In [1]:
!pip install flask
!pip install ipywidgets


In [2]:
import math
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [3]:
diabetes_train = pd.read_csv("https://raw.githubusercontent.com/icesaglo/thenoncomps/refs/heads/main/diabetes_train.csv")
diabetes_test = pd.read_csv("https://raw.githubusercontent.com/icesaglo/thenoncomps/refs/heads/main/diabetes_test.csv")

In [ ]:
from flask import Flask, render_template_string, request

# Initialize the Flask app
app = Flask(__name__)

# Define the HTML template for the form
form_template = '''
<!doctype html>
<html lang="en">
  <head>
    <title>Input Form</title>
    
    <style>
      body {
        font-family: Arial, sans-serif;
        background-color: #f0f8ff; /* Light blue background */
        color: #333; /* Dark text color */
        margin: 0;
        padding: 20px;
      }
      h2 {
        color: #005f73; /* Dark teal for heading */
      }
      form {
        background-color: #fff; /* White background for form */
        padding: 20px;
        border-radius: 8px;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        max-width: 400px;
        margin: 0 auto;
      }
      label {
        display: block;
        margin-bottom: 8px;
        color: #0a9396; /* Medium teal for labels */
      }
      input[type="number"] {
        width: 100%;
        padding: 8px;
        margin-bottom: 16px;
        border: 1px solid #ccc;
        border-radius: 4px;
      }
      input[type="submit"] {
        background-color: #94d2bd; /* Light green for submit button */
        color: white;
        border: none;
        padding: 10px 15px;
        cursor: pointer;
        border-radius: 4px;
      }
      input[type="submit"]:hover {
        background-color: #0a9396; /* Darker teal on hover */
      }
      h3 {
        color: #005f73; /* Dark teal for results heading */
        margin-top: 30px;
      }
      p {
        color: #333; /* Dark text color for results */
      }
    </style>
  </head>
  <body>
    <h2>Diabetes prediction</h2>
    <form method="POST" action="/">
      <label for="age">Age:</label><br>
      <input type="number" id="age" name="age" required><br><br>
      
      <label for="height">Height (cm):</label><br>
      <input type="number" id="height" name="height" required><br><br>
      
      <label for="weight">Weight (kg):</label><br>
      <input type="number" id="weight" name="weight" required><br><br>

      <label for="pregnancies">Number of times pregnant:</label><br>
      <input type="number" id="pregnancies" name="pregnancies" required><br><br>

      <label for="glucose">Plasma glucose concentration (mg/dL):</label><br>
      <input type="number" id="glucose" name="glucose" required><br><br>
      
      <label for="BP">Blood Pressure (mmHg):</label><br>
      <input type="number" id="BP" name="BP" required><br><br>

      <label for="skin">Skin Thickness (mm):</label><br>
      <input type="number" id="skin" name="skin" required><br><br>

      <label for="insulin">Insulin (pmol/L):</label><br>
      <input type="number" id="insulin" name="insulin" required><br><br>
    

      <input type="submit" value="Submit">
    </form>

    {% if bmi %}
      <h3>Your Results</h3>
      <p>Age: {{ age }} years</p>
      <p>Height: {{ height }} cm</p>
      <p>Weight: {{ weight }} kg</p>
      <p>BMI: {{ bmi }}</p>
      <p>Number of Pregnancies: {{ pregnancies }}</p>
      <p>Glucose Concentration: {{ glucose }} mg/dL</p>
      <p>Blood Pressure: {{ BP }} mmHg</p>
      <p>Skin Thickness: {{ skin }} mm</p>
      <p>Insulin: {{ insulin }} pmol/L</p>
      <p>risk: {{risk}} </p>
    {% endif %}
  </body>
</html>
'''

# Function to calculate BMI
def calculate_bmi(weight, height):
    if height > 0:
        height_in_meters = height / 100  # Convert height to meters
        return round(weight / (height_in_meters ** 2), 2)
    return None

# random forest
def func(diabetes_train):
    X_train, y_train = diabetes_train.iloc[:, 1:9], diabetes_train.iloc[:, 9]
    
    rf_classifier = RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=10, n_estimators=200, random_state=205)
    rf_classifier.fit(X_train, y_train)
    
    return rf_classifier

def predict(var_list):
    rf_classifier = func(diabetes_train)
    var_list = np.array(var_list).reshape(1, -1) 
    risk = rf_classifier.predict_proba(var_list)
    return risk[0][0]

# Route for the form and input handling
@app.route("/", methods=["GET", "POST"])
def index():
    bmi = None
    age = None
    height = None
    weight = None
    pregnancies = None
    glucose = None
    BP = None
    skin = None
    insulin = None
    risk = None
    if request.method == "POST":
        age = int(request.form["age"])
        height = float(request.form["height"])
        weight = float(request.form["weight"])
        pregnancies = int(request.form["pregnancies"])
        glucose = int(request.form["glucose"])
        BP = int(request.form["BP"])
        skin = int(request.form["skin"])
        insulin = int(request.form["insulin"])

        # Calculate BMI
        bmi = calculate_bmi(weight, height)

        var_list = [math.log(pregnancies + 1), math.log(glucose), BP, skin, math.log(insulin), bmi, 0.4, age]
        risk = predict(var_list)
        
    return render_template_string(form_template, bmi=bmi, age=age, height=height, weight=weight, pregnancies=pregnancies, glucose=glucose, BP=BP, skin=skin, insulin = insulin, risk=risk)

# Run the app
if __name__ == '__main__':
    app.run()
   

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Oct/2024 22:51:33] "GET / HTTP/1.1" 200 -
C:\Users\ngwk0\Jupyter\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [19/Oct/2024 22:52:04] "POST / HTTP/1.1" 200 -
